In [ ]:
# --- Training notebook: EfficientNet-B1 + Optuna tuning + final training ---

import os
import random
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision import models
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import optuna
from tqdm.notebook import tqdm

# --- Configuration ---
BASE_INPUT_PATH = '/kaggle/input/soil-classification/soil_classification-2025'
TRAIN_DIR = os.path.join(BASE_INPUT_PATH, 'train')
TEST_DIR = os.path.join(BASE_INPUT_PATH, 'test')
TRAIN_LABELS_CSV = os.path.join(BASE_INPUT_PATH, 'train_labels.csv')
TEST_IDS_CSV = os.path.join(BASE_INPUT_PATH, 'test_ids.csv')
OUTPUT_DIR = '/kaggle/working/'

IMG_SIZE = 240
NUM_CLASSES = 4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

N_OPTUNA_TRIALS = 25
EPOCHS_PER_TRIAL = 12
FINAL_MODEL_EPOCHS = 30
STUDY_SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(STUDY_SEED)

# 1. Load Data & EDA
df_train = pd.read_csv(TRAIN_LABELS_CSV)
df_test_ids = pd.read_csv(TEST_IDS_CSV)
class_names = sorted(df_train['soil_type'].unique())
label_to_int = {lbl:i for i,lbl in enumerate(class_names)}
int_to_label = {i:lbl for lbl,i in label_to_int.items()}
df_train['label_int'] = df_train['soil_type'].map(label_to_int)
print("Class distribution:\n", df_train['soil_type'].value_counts())

# 2. Preprocessing & Transforms
mean, std = [0.485,0.456,0.406], [0.229,0.224,0.225]
train_transforms = T.Compose([
    T.Resize((IMG_SIZE,IMG_SIZE)),
    T.RandomHorizontalFlip(), T.RandomVerticalFlip(),
    T.RandomRotation(45),
    T.ColorJitter(0.3,0.3,0.3,0.15),
    T.RandomAffine(0, translate=(0.1,0.1), scale=(0.9,1.1), shear=10),
    T.ToTensor(), T.Normalize(mean,std)
])
val_transforms = T.Compose([
    T.Resize((IMG_SIZE,IMG_SIZE)),
    T.ToTensor(), T.Normalize(mean,std)
])

class SoilDataset(Dataset):
    def __init__(self, df, image_dir, transforms=None, is_test=False):
        self.df = df
        self.dir = image_dir
        self.transforms = transforms
        self.is_test = is_test
        if not is_test:
            self.labels = df['label_int'].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.dir, row['image_id'])
        try:
            img = Image.open(img_path).convert('RGB')
        except:
            img = Image.new('RGB',(IMG_SIZE,IMG_SIZE),color='black')
        if self.transforms:
            img = self.transforms(img)
        if self.is_test:
            return img, row['image_id']
        return img, torch.tensor(self.labels[idx], dtype=torch.long)

# 3. Split train/val
train_df, val_df = train_test_split(
    df_train, test_size=0.2,
    stratify=df_train['label_int'],
    random_state=STUDY_SEED
)
train_ds = SoilDataset(train_df, TRAIN_DIR, train_transforms)
val_ds   = SoilDataset(val_df,   TRAIN_DIR, val_transforms)

# 4. Optuna objective
def objective(trial):
    lr = trial.suggest_float("lr",1e-5,1e-3,log=True)
    opt_name = trial.suggest_categorical("optimizer",["AdamW","Adam"])
    bs = trial.suggest_categorical("batch_size",[16,32])
    wd = trial.suggest_float("weight_decay",1e-6,1e-2,log=True)
    unfreeze = trial.suggest_int("num_unfreeze_blocks",1,4)
    sched_pat = trial.suggest_int("sched_patience",2,4)
    sched_fac = trial.suggest_float("sched_factor",0.1,0.5)

    # DataLoaders
    try:
        train_loader = DataLoader(train_ds, batch_size=bs, shuffle=True,
                                  num_workers=2, pin_memory=True, drop_last=True)
        val_loader   = DataLoader(val_ds, batch_size=bs, shuffle=False,
                                  num_workers=2, pin_memory=True)
    except RuntimeError as e:
        if "out of memory" in str(e).lower():
            torch.cuda.empty_cache()
            raise optuna.TrialPruned()
        raise

    # Model setup
    model = models.efficientnet_b1(weights=models.EfficientNet_B1_Weights.IMAGENET1K_V1)
    for p in model.parameters(): p.requires_grad=False
    in_f = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(in_f, NUM_CLASSES)
    for p in model.classifier.parameters(): p.requires_grad=True
    total = len(model.features)
    for i in range(total-unfreeze, total):
        for p in model.features[i].parameters():
            p.requires_grad=True
    model.to(DEVICE)

    optimizer = (optim.AdamW if opt_name=="AdamW" else optim.Adam)(
        [p for p in model.parameters() if p.requires_grad],
        lr=lr, weight_decay=wd
    )
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=sched_fac, patience=sched_pat, verbose=False
    )
    criterion = nn.CrossEntropyLoss()

    best_min_f1 = 0.0
    for epoch in range(EPOCHS_PER_TRIAL):
        model.train()
        for X,y in train_loader:
            X,y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(model(X), y)
            loss.backward()
            optimizer.step()

        model.eval()
        preds, labs = [], []
        with torch.no_grad():
            for X,y in val_loader:
                X,y = X.to(DEVICE), y.to(DEVICE)
                _, p = model(X).max(1)
                preds += p.cpu().tolist()
                labs += y.cpu().tolist()

        f1s = f1_score(labs, preds, average=None,
                       labels=list(range(NUM_CLASSES)), zero_division=0)
        cur_min = float(np.min(f1s))
        best_min_f1 = max(best_min_f1, cur_min)
        scheduler.step(cur_min)

        trial.report(cur_min, epoch)
        if trial.should_prune():
            torch.cuda.empty_cache()
            raise optuna.TrialPruned()

    torch.cuda.empty_cache()
    return best_min_f1

# 5. Run study & final retrain
pruner = optuna.pruners.SuccessiveHalvingPruner(min_resource=1, reduction_factor=4)
study = optuna.create_study(direction="maximize", pruner=pruner, study_name="soil_effnet_b1")
study.optimize(objective, n_trials=N_OPTUNA_TRIALS, timeout=10800)
best = study.best_trial.params

# Final model retraining
train_loader = DataLoader(train_ds, batch_size=best['batch_size'], shuffle=True,
                          num_workers=2, pin_memory=True, drop_last=True)
val_loader   = DataLoader(val_ds,   batch_size=best['batch_size'], shuffle=False,
                          num_workers=2, pin_memory=True)

model_fin = models.efficientnet_b1(weights=models.EfficientNet_B1_Weights.IMAGENET1K_V1)
for p in model_fin.parameters(): p.requires_grad=False
in_f = model_fin.classifier[1].in_features
model_fin.classifier[1] = nn.Linear(in_f, NUM_CLASSES)
for p in model_fin.classifier.parameters(): p.requires_grad=True
for i in range(len(model_fin.features)-best['num_unfreeze_blocks'], len(model_fin.features)):
    for p in model_fin.features[i].parameters():
        p.requires_grad=True
model_fin.to(DEVICE)

optimizer_fin = (optim.AdamW if best['optimizer']=="AdamW" else optim.Adam)(
    [p for p in model_fin.parameters() if p.requires_grad],
    lr=best['lr'], weight_decay=best['weight_decay']
)
scheduler_fin = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer_fin, mode='max', factor=best['sched_factor'],
    patience=best['sched_patience'], verbose=True
)
criterion_fin = nn.CrossEntropyLoss()

best_val_f1 = 0.0
for epoch in range(FINAL_MODEL_EPOCHS):
    model_fin.train()
    for X,y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{FINAL_MODEL_EPOCHS}"):
        X,y = X.to(DEVICE), y.to(DEVICE)
        optimizer_fin.zero_grad()
        loss = criterion_fin(model_fin(X), y)
        loss.backward()
        optimizer_fin.step()

    model_fin.eval()
    preds, labs = [], []
    with torch.no_grad():
        for X,y in val_loader:
            X,y = X.to(DEVICE), y.to(DEVICE)
            _, p = model_fin(X).max(1)
            preds += p.cpu().tolist()
            labs += y.cpu().tolist()

    f1s = f1_score(labs, preds, average=None,
                   labels=list(range(NUM_CLASSES)), zero_division=0)
    cur_min = float(np.min(f1s))
    print(f"Epoch {epoch+1} min F1: {cur_min:.4f}", f1s)
    scheduler_fin.step(cur_min)

    if cur_min > best_val_f1:
        best_val_f1 = cur_min
        torch.save(model_fin.state_dict(), os.path.join(OUTPUT_DIR, "best_effnet_b1.pth"))

print("Training complete, best min-class F1:", best_val_f1)
